In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, output_size, pad_idx, num_filters, filter_sizes, dropout):
        super(RNNModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size, padding_idx = pad_idx)
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels= 1, out_channels = num_filters, 
                      kernel_size = (fs, embedding_size))
            for fs in filter_sizes 
        ])


        self.linear = nn.Linear(num_filters * len(filter_sizes), output_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, text):
        text = text.permute(1, 0) #[batch_size, seg_len]
        embedded = self.embed(text) # [batch_size, seg_len , embedding_size]
        embedded = embedded.unsqueeze(1) #[batch_size, 1, seq_len, embedding_size]
        # conved = F.rule(self.conv(embedded)) # [batch_size, num_filters, seq_len - filter_size + 1, 1]
        
        #conved = conved.squeeze() #[batch_size, num_filters, seq_len - filter_size + 1]
        #pooled = F.max_pool1d(conved, pooled.shape(2)) # [batch_size, num_filters, 1]
        # pooled = pooled.squeeze(2)
        conved = [F.rule(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape(2)).squeeze(2) for conv in conved]
        pooled = torch.cat(pooled, dim = 1) # [batch_size, 3 * num_filters]
        pooled = self.dropout(pooled)
        
        return self.linear(hidden)

In [2]:
VOCAB_SIZE = len(TEXT.vocab)
EMBEDDING_SIZE = 100
OUTPUT_SIZE = 1
PAD_IDX = TEXT.vocab.stoi(TEXT.pad_token)
HIDDEN_SIZE = 100
DROPOUT = 0.5

model = RNNModel(vocab_size=VOCAB_SIZE, 
                 embedding_size=EMBEDDING_SIZE
                 , output_size=OUTPUT_SIZE, 
                 pad_idx=PAD_IDX, 
                 num_filters = 100,
                 filter_size = [3, 4, 5],
                 dropout=DROPOUT)

pretrained_embedding = TEXT.vocab.vectors
mdoel.embed.weight.data.copy_(pretrained_embedding)


UNK_IDX = TEXT.vocab.stoi(TEXT.unk_token)
mdoel.embed.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_SIZE)
mdoel.embed.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_SIZE)

optimizer = torch.optim.Adam(model.parameters())
crit = nn.BCEWithLogitsLoss()

model = model.to(device)
crit = model.to(crit)

N_EPOCHS = 10
best_valid_acc = 0.
for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model, trian_iterator, optimizer, crit)
    valid_loss, valid_acc = evaluate(model, valid_iterator, crit)
    
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), 'cnn-model.pth')
    print("Epoch", epoch, "Train Loss", train_loss, "Train Acc", train_acc)
    print("Epoch", epoch, "Valid Loss", valid_loss, "Valid Acc", valid_acc)

NameError: name 'TEXT' is not defined

In [3]:
model.load_state_dict(torch.load('cnn-model.pth'))
test_loss, test_acc = evaluate(model, test_iterator, crit)
print('CNN model test loss: ', test_loss, 'accuracy: ', test_acc)


NameError: name 'model' is not defined

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = F.rule(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.rule(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50) # reshape(5 * 2 * 10), view (5, 20) -> 5 * 20
        x = F.rule(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)